# **Sieci Neuronowe** (laboratorium 1/5)

## Zasady zaliczenia
1. **Obecność na laboratoriach:**
   - Obecność na zajęciach laboratoryjnych jest obowiązkowa.
   - Obecność jest potwierdzana poprzez wysłanie rozwiązania zadania przez UPEL.
   - Każda nieobecnoś USPRAWIEDLIWIONA należy odrobić, najszybciej jak to jest możliwe. W przeciwnym wypadku student/studentka uzyskują 0pkt.
   - Każda obecność nieusprawiedliwiona obniża ocenę końcową 1 stopień.

2. **Odrabianie zajęć:**
   - Zajęcia można odrobić w innej grupie laboratoryjnej, jeśli są dostępne miejsca.
   - Pierwszeństwo w uczestnictwie mają osoby przypisane do danej grupy.
   - Chęć odrabiania proszę zakomunikować przynajmniej dzień wcześniej.

3. **Terminy odrabiania zajęć:**
   - Odrabianie zajęć może odbywać się tylko w tygodniu, w którym dany temat jest analizowany.
   - Student/Studentka zobowiązany jest zgłosić się do prowadzącego zajęcia w celu ustalenia terminu odrobienia zajęć.

4. **Brak usprawiedliwionej nieobecności:**
   - Student opuszczający więcej niż 50% zajęć bez usprawiedliwienia i uzyskujący negatywne wyniki w nauce może stracić możliwość wyrównania zaległości.
   - Student może odwołać się od decyzji prowadzącego zajęcia do prowadzącego przedmiot i/lub Dziekana.

6. **Usprawiedliwienia:**
   - Usprawiedliwienia należy przesyłać w formie skanu na maila prowadzącego na adres: krzywda@agh.edu.pl, w temacie "[SSN ##] Usprawiedliwienie", gdzie ## to numer grupy.

7. **Ocena laboratorium:**
   - Ocena laboratorium będzie opierać się na ilości zdobytych punktów za sprawozdania. Sprawozdania wysyłacie Państwo na UPEL.
   - Po każdych zajęciach należy przesłać sprawozdanie w formie notatnika .ipynb, w ramach którego umieszcza się komórki z kodem, wykresami oraz komórki z wnioskami,tekstem (markdown)
   - Każde laboratorium ocenianie jest w skali 0-100. Żeby uzuskać zaliczenia należy uzyskać minimum **50%** punktów.
   - Sprawozdań nie można poprawiać.
   - Sprawozdanie można wysłać do dnia poprzedzającego kolejne zajęcia.
   - Sprawozdania wysłane do tygodnia po terminie są oceniane w skali 0-50 (gdzie 50pkt to max za wykonanie wszystkich zadań). Po dwóch tygodniach max 25pkt itd.
   - Ocena końcowa jest obliczana według zasad ogólnych AGH. Czyli:
    - < 50.00% puktów to 2.0  
    - 50.00-59.99% punktów to 3.0
    - 60.00 -69.99% to 3.5
    - 70.00 -79.99% to 4.0
    - 80.00 - 89.99% to 4.5
    - 90.00 % to 5.0
    
    Przykład:
    Student/Studentka przesłała 3 sprawozdania, wszystkie idealne, na 100pkt oraz dwa przesyła tydzień po terminie to 2x50pkt. Wiec to łacznie 400pkt/500 ->

8. **Tematyka zagadnień**
    - Temat 1: Convolutional Neural Network cz. I
    - Temat 2: Convolutional Neural Network cz. II
    - Temat 3: Convolutional Neural Network cz. III + Recurrent Neural Network cz. I
    - Temat 4: Recurrent Neural Network cz. II
    - Temat 5: Graph Neural Networks

9. **Technologie**
    - PyTorch
    - Google Colab


# **Wprowadzenie**

## Omówienie kodu klasyfikacji obrazów z użyciem biblioteki PyTorch
### Importowanie bibliotek
```python
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
```

Importowanie niezbędnych bibliotek: `torch` do obliczeń tensorowych, `torch.nn` do budowy sieci neuronowych, `torch.optim` do optymalizacji, a także moduł `torchvision` do pracy z zestawami danych obrazowych i transformacjami.



## Przygotowanie zestawu danych treningowych i testowych

```python
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)
```

Przygotowanie transformacji dla zestawów treningowych i testowych, w tym losowe przycinanie, odbicia horyzontalne i normalizacja. Następnie pobranie zestawu danych CIFAR-10 i utworzenie obiektów DataLoader do ładowania danych treningowych i testowych partiami.



## Definicja modelu sieci neuronowej

```python
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Definicja pierwszej warstwy konwolucyjnej: wejście 3 kanały (RGB), wyjście 32 kanały, rozmiar filtra 3x3, padding=1
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        # Definicja drugiej warstwy konwolucyjnej: wejście 32 kanały, wyjście 64 kanały, rozmiar filtra 3x3, padding=1
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        # Definicja trzeciej warstwy konwolucyjnej: wejście 64 kanały, wyjście 128 kanały, rozmiar filtra 3x3, padding=1
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        # Warstwa dropout1 z prawdopodobieństwem wyzerowania 25% neuronów
        self.dropout1 = nn.Dropout(0.25)
        # Warstwa dropout2 z prawdopodobieństwem wyzerowania 50% neuronów
        self.dropout2 = nn.Dropout(0.5)
        # Warstwa w pełni połączona (fully connected), wejście 128 * 4 * 4 neurony (wynik rozmiaru tensora po ostatniej konwolucji), wyjście 512 neurony
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        # Warstwa w pełni połączona (fully connected), wejście 512 neurony, wyjście 10 neurony (liczba klas)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        # Przekształcenie wejścia przez pierwszą warstwę konwolucyjną, zastosowanie funkcji aktywacji ReLU
        x = nn.functional.relu(self.conv1(x))
        # Redukcja wymiarowości przez operację max-pooling z rozmiarem okna 2x2
        x = nn.functional.max_pool2d(x, 2)
        # Przekształcenie przez drugą warstwę konwolucyjną, zastosowanie funkcji aktywacji ReLU
        x = nn.functional.relu(self.conv2(x))
        # Redukcja wymiarowości przez operację max-pooling z rozmiarem okna 2x2
        x = nn.functional.max_pool2d(x, 2)
        # Przekształcenie przez trzecią warstwę konwolucyjną, zastosowanie funkcji aktywacji ReLU
        x = nn.functional.relu(self.conv3(x))
        # Redukcja wymiarowości przez operację max-pooling z rozmiarem okna 2x2
        x = nn.functional.max_pool2d(x, 2)
        # Wykorzystanie warstwy dropout1
        x = self.dropout1(x)
        # Spłaszczenie tensora do postaci wektora przed podaniem na warstwę w pełni połączoną
        x = torch.flatten(x, 1)
        # Przekształcenie przez warstwę w pełni połączoną, zastosowanie funkcji aktywacji ReLU
        x = nn.functional.relu(self.fc1(x))
        # Wykorzystanie warstwy dropout2
        x = self.dropout2(x)
        # Przekształcenie przez warstwę w pełni połączoną (wyjściową)
        x = self.fc2(x)
        # Zastosowanie funkcji log_softmax dla uzyskania prawdopodobieństw przynależności do klas
        output = nn.functional.log_softmax(x, dim=1)
        return output

```

Definicja klasy modelu sieci neuronowej. Warstwy konwolucyjne (Conv2d), warstwy Dropout (Dropout), oraz warstwy liniowe (Linear). Metoda forward definiuje przepływ danych przez sieć.

## Definicja funkcji straty i optymalizatora

```python
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
```

Inicjalizacja modelu, funkcji straty (CrossEntropyLoss) oraz optymalizatora (SGD) z odpowiednimi parametrami.




## Trenowanie modelu

```python
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0
```

Pętla trenowania modelu przez 10 epok. Dla każdej epoki obliczane są straty na zestawie treningowym, propagowane wstecz i aktualizowane wagi sieci.



## Testowanie modelu
```python
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")


```

W **PyTorch** istnieje wiele przydatnych funkcji i warstw, które są wykorzystywane do budowy sieci neuronowych. Kilka z nich to:

- [Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html): Warstwa konwolucyjna dwuwymiarowa, używana do przetwarzania danych wizyjnych.

- [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html): Warstwa dropout, która losowo zeruje część wejść w celu regularyzacji i zapobiegania przeuczeniu.

- [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html): Warstwa w pełni połączona, używana do transformacji liniowej danych.

Ponadto, w module `torch.nn.functional` znajdują się różne funkcje aktywacji i operacje, takie jak:

- [relu](https://pytorch.org/docs/stable/generated/torch.nn.functional.relu.html): Funkcja aktywacji ReLU (Rectified Linear Unit), używana do wprowadzenia nieliniowości w sieciach neuronowych.

- [max_pool2d](https://pytorch.org/docs/stable/generated/torch.nn.functional.max_pool2d.html): Operacja max-pooling dla danych dwuwymiarowych, używana do redukcji wymiarowości.

- [log_softmax](https://pytorch.org/docs/stable/generated/torch.nn.functional.log_softmax.html): Funkcja aktywacji logarytmicznej softmax, używana do obliczania logarytmów prawdopodobieństw przynależności do klas.


W PyTorch znajduje się wiele optymalizatorów, funkcji straty oraz funkcji aktywacji, które są powszechnie używane podczas trenowania sieci neuronowych. Kilka z nich to:

Optymalizatory:

- [SGD](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD): Stochastyczny spadek gradientu, podstawowy optymalizator wykorzystywany do aktualizacji wag sieci.

- [Adam](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam): Algorytm optymalizacji gradientowej stochastycznej, który adaptuje współczynniki uczenia dla każdego parametru.

- [RMSprop](https://pytorch.org/docs/stable/optim.html#torch.optim.RMSprop): Algorytm optymalizacji gradientowej stochastycznej, który korzysta z średniej ważonej kwadratów gradientów, aby adaptować współczynnik uczenia.

Funkcje straty:

- [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html): Funkcja straty krzyżowej entropii, często używana w zadaniach klasyfikacji wieloklasowej.

- [MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html): Funkcja straty błędu średniokwadratowego, często używana w zadaniach regresji.

Funkcje aktywacji:

- [ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html): Funkcja aktywacji ReLU (Rectified Linear Unit), wprowadza nieliniowość do modelu poprzez zastosowanie wartości maksymalnej z 0 i wartości x.

- [Sigmoid](https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html): Funkcja aktywacji sigmoidalna, której wartości wyjściowe mieszczą się w zakresie (0, 1), często używana w ostatniej warstwie modelu do zadania binarnej klasyfikacji.

- [Tanh](https://pytorch.org/docs/stable/generated/torch.nn.Tanh.html): Funkcja aktywacji tangens hiperboliczny, przekształca wartości wejściowe do zakresu (-1, 1).


**Metryki** te są wykorzystywane do oceny jakości predykcji modelu podczas procesu treningu oraz testowania.

1. Accuracy - metryka ta mierzy procent poprawnych klasyfikacji w zbiorze danych.

2. Binary accuracy - metryka ta mierzy procent poprawnych klasyfikacji dla problemów binarnych.

3. Categorical accuracy - metryka ta mierzy procent poprawnych klasyfikacji dla problemów wieloklasowych.

4. Precision - metryka ta mierzy proporcję poprawnie sklasyfikowanych przykładów pozytywnych do ogólnej liczby sklasyfikowanych pozytywnie.

5. Recall - metryka ta mierzy proporcję poprawnie sklasyfikowanych przykładów pozytywnych do ogólnej liczby prawdziwych pozytywnych.

6. F1 score - metryka ta jest średnią harmoniczną precyzji i recall, a więc uwzględnia zarówno false positives, jak i false negatives.

7. Mean squared error (MSE) - metryka ta mierzy średnią kwadratową różnicę między predykcjami a rzeczywistymi wartościami.

8. Root mean squared error (RMSE) - metryka ta mierzy pierwiastek średniej kwadratowej różnicy między predykcjami a rzeczywistymi wartościami.

9. Mean absolute error (MAE) - metryka ta mierzy średnią wartość bezwzględną różnicy między predykcjami a rzeczywistymi wartościami.

### Warstwa konwolucyjna / splotowa (Convolutional Layers)

Warstwa konwolucyjna, znana również jako warstwa splotowa, jest podstawowym elementem sieci neuronowych wykorzystywanych głównie w przetwarzaniu obrazów. Polega na przetwarzaniu sygnału wejściowego za pomocą filtrów konwolucyjnych, które przesuwane są po obrazie, generując mapy cech.

`torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, ...)`

- `in_channels`: Liczba kanałów wejściowych.
- `out_channels`: Liczba kanałów wyjściowych (rozmiar głębokości mapy cech).
- `kernel_size`: Rozmiar jądra (filtru).
- `stride`: Krok przesunięcia filtra (domyślnie 1).
- `padding`: Wypełnienie obrazu (domyślnie 0).
- Inne parametry dostępne w warstwie konwolucyjnej w PyTorch.

### Conv1D, Conv2D, Conv3D w PyTorch

W PyTorch warstwy konwolucyjne dostępne są w różnych wymiarach przestrzennych:

- **Conv1D**: Wykorzystywana głównie do przetwarzania szeregów czasowych.
- **Conv2D**: Stosowana przede wszystkim w przetwarzaniu obrazów.
- **Conv3D**: Używana w przypadku danych trójwymiarowych, np. wideo.

### Warstwa poolingu

Warstwa poolingu jest kolejnym elementem stosowanym w sieciach neuronowych, służącym do zmniejszania wymiarowości danych poprzez agregację informacji. Wyróżnia się kilka rodzajów warstw poolingu:

- **Max Pooling**: Wybiera największą wartość z określonego obszaru.
- **Average Pooling**: Oblicza średnią wartość z określonego obszaru.
- **Global Pooling**: Agreguje informacje na poziomie całego obrazu lub wolumenu.
- **Min Pooling**: Wybiera najmniejszą wartość z określonego obszaru.

W PyTorch warstwy poolingu można zaimplementować za pomocą odpowiednich funkcji, np. `torch.nn.MaxPool2d`, `torch.nn.AvgPool2d`, itp.


### Batch Normalization w PyTorch

Batch Normalization (BN) to technika stosowana w sieciach neuronowych w celu poprawy szybkości uczenia się, stabilności i wydajności. W PyTorch Batch Normalization jest dostępna jako moduł `torch.nn.BatchNorm*d*`, gdzie `*d*` oznacza wymiar danych wejściowych (1D, 2D, itd.).

#### Działanie Batch Normalization

1. **Normalizacja Batch**: Dla każdej mini-batch danych, obliczane są średnia i odchylenie standardowe.
2. **Normalizacja**: Dane wejściowe są normalizowane na podstawie obliczonych wartości średniej i odchylenia standardowego.
3. **Skalowanie i Przesunięcie**: Dane są skalowane i przesuwane za pomocą dodatkowych parametrów w celu zachowania elastyczności modelu.

#### Parametry Warstwy Batch Normalization

- `num_features`: Liczba cech (kanałów) wejściowych.
- `eps`: Parametr zapobiegający dzieleniu przez zero w przypadku niskiego odchylenia standardowego.
- `momentum`: Parametr kontrolujący skalowanie wcześniejszej wartości średniej i odchylenia standardowego.

#### Implementacja w PyTorch

```python
# Definicja modelu
model = nn.Sequential(
    nn.Linear(input_size, num_features),
    nn.BatchNorm1d(num_features),  # BatchNorm1d dla danych 1D
    nn.ReLU(),
    nn.Linear(num_features, 1)
)


### Regularyzacja w PyTorch

Regularyzacja jest techniką stosowaną w uczeniu maszynowym w celu zapobiegania nadmiernemu dopasowaniu (przeuczeniu) modelu do danych treningowych. W PyTorch można zaimplementować różne techniki regularizacji, takie jak L1, L2, Dropout itp.

#### L1 i L2 Regularyzacja

Regularyzacja L1 i L2 to popularne metody regularyzacji, które dodają kary do funkcji kosztu modelu na podstawie normy L1 lub L2 wag. W PyTorch regularyzacja L1 i L2 jest zazwyczaj implementowana poprzez dodanie kary do funkcji kosztu w trakcie treningu.

```python
import torch
import torch.nn as nn
import torch.optim as optim

# Przykładowa implementacja regularyzacji L2
model = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, output_size)
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=0.001)  # weight_decay to parametr regularyzacji L2

# W trakcie treningu
for inputs, labels in train_loader:
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
```

#### Dropout
Dropout jest techniką regularizacji, która polega na losowym wyłączaniu neuronów w trakcie treningu, co zmusza model do bardziej równomiernego wykorzystania wszystkich cech. W PyTorch można zaimplementować warstwę Dropout za pomocą torch.nn.Dropout.

```python
# Przykładowa implementacja Dropout
model = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    nn.Dropout(p=0.5),  # p to prawdopodobieństwo wyłączenia neuronu
    nn.Linear(hidden_size, output_size)
)
````

Uwagi
- Regularyzacja pomaga w zapobieganiu przeuczeniu modelu.
- Odpowiedni dobór parametrów regularyzacji może poprawić ogólną zdolność generalizacji modelu.
- Warto eksperymentować z różnymi technikami regularizacji i parametrami, aby uzyskać najlepsze rezultaty.

# **Materiały do zapoznania się**
https://poloclub.github.io/cnn-explainer/
- https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53
- https://www.freecodecamp.org/news/an-intuitive-guide-to-convolutional-neural-networks-260c2de0a050/
- https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d
- https://cv-tricks.com/cnn/understand-resnet-alexnet-vgg-inception/
- https://medium.com/analytics-vidhya/cnns-architectures-lenet-alexnet-vgg-googlenet-resnet-and-more-666091488df5
- http://yann.lecun.com/exdb/lenet/

Best practices to build neural network models:

* https://medium.com/@dipti.rohan.pawar/improving-performance-of-convolutional-neural-network-2ecfe0207de7

* https://medium.com/towards-data-science/a-guide-to-an-efficient-way-to-build-neural-network-architectures-part-i-hyper-parameter-8129009f131b

* https://towardsdatascience.com/a-guide-to-an-efficient-way-to-build-neural-network-architectures-part-ii-hyper-parameter-42efca01e5d7



Artykuły i snippety kodu + podstawowe zagadnienia do CNN

* Idea konwolucji: http://sciagaprogramisty.blogspot.com/2018/01/konwolucja-wstep-do-neuronowych-sieci.html

* NN architectures: https://pub.towardsai.net/main-types-of-neural-networks-and-its-applications-tutorial-734480d7ec8e?gi=28a8f0c56697

* https://www.kaggle.com/code/cdeotte/how-to-choose-cnn-architecture-mnist/notebook

* https://medium.com/analytics-vidhya/a-guide-to-neural-network-layers-with-applications-in-keras-40ccb7ebb57a

* https://towardsdatascience.com/a-quick-guide-to-neural-network-optimizers-with-applications-in-keras-e4635dd1cca4

* https://medium.com/@andre_ye/a-quick-guide-to-neural-network-optimizers-with-applications-in-keras-e4635dd1cca4

* https://towardsdatascience.com/a-guide-to-neural-network-loss-functions-with-applications-in-keras-3a3baa9f71c5




Keras vs Tensorflow vs Pytorch



* https://karliris62.medium.com/keras-vs-tensorflow-which-one-is-the-right-fit-for-your-project-7a166fe6c64b

* https://www.edureka.co/blog/keras-vs-tensorflow-vs-pytorch/

* https://towardsdatascience.com/batch-normalization-the-greatest-breakthrough-in-deep-learning-77e64909d81d





# Zadanie 1

Po zapoznaniu się z obsluga interfejsu Google Colab. **Zaimplementuj** trzy wielowarstwowe Convolutional Neural Networks (CNNs) do klasyfikacji datasetów:
* [MNIST](https://paperswithcode.com/dataset/mnist)
* [Fashion MNIST](https://paperswithcode.com/dataset/fashion-mnist)
* [CIFAR-100](https://paperswithcode.com/sota/image-classification-on-cifar-100)

W tym celu wykorzystaj biblioteke **PyTorch**.

W sprawozdaniu odpowiedz na poniższe pytania opierając się o uzyskane wyniki w trakcie wykonywania eksperymentów.
- Jakich warstw użyłeś/użyłaś w swoim modelu? Dlaczego? Dlaczego dobrałeś/dobrałaś takie parametry?
- Narysuj przebieg funkcji straty oraz wybranych dwóch innych metryk (`matplotlib` albo wykorzystaj wbudowane funkcje w `PyTorch`). Co z nich wynika? Czym jest `overfitting` czym jest `underfitting`? Jaki mechanizm można zastosować żeby uniknać przeuczenia? Jak uniknąć niedouczenia?
- Jakich optymalizatorów oraz funkcji straty użyłeś/aś? Dlaczego? Jakich parametrów? Za co odpowiada optymalizator w procesie uczenia?
- Każdy model, dla każdego datasetu uruchom 5razy. Uśrednij wyniki i przedstaw to w formie tabelki(może być tabelka dataframe z `pandas` i wykorzystać `from tabulate import tabulate` do wyświetlenia tego w ładniejszy sposób).
- Przygotuj macierz pomyłek dla każdego swojego modelu.